--- Day 23: Crab Cups ---

The small crab challenges you to a game! The crab is going to mix up some cups, and you have to predict where they'll end up.

The cups will be arranged in a circle and labeled clockwise (your puzzle input). For example, if your labeling were 32415, there would be five cups in the circle; going clockwise around the circle from the first cup, the cups would be labeled 3, 2, 4, 1, 5, and then back to 3 again.

Before the crab starts, it will designate the first cup in your list as the current cup. The crab is then going to do 100 moves.

Each move, the crab does the following actions:

*    The crab picks up the three cups that are immediately clockwise of the current cup. They are removed from the circle; cup spacing is adjusted as necessary to maintain the circle.
*    The crab selects a destination cup: the cup with a label equal to the current cup's label minus one. If this would select one of the cups that was just picked up, the crab will keep subtracting one until it finds a cup that wasn't just picked up. If at any point in this process the value goes below the lowest value on any cup's label, it wraps around to the highest value on any cup's label instead.
*    The crab places the cups it just picked up so that they are immediately clockwise of the destination cup. They keep the same order as when they were picked up.
*    The crab selects a new current cup: the cup which is immediately clockwise of the current cup.

For example, suppose your cup labeling were 389125467. If the crab were to do merely 10 moves, the following changes would occur:

```
-- move 1 --
cups: (3) 8  9  1  2  5  4  6  7 
pick up: 8, 9, 1
destination: 2

-- move 2 --
cups:  3 (2) 8  9  1  5  4  6  7 
pick up: 8, 9, 1
destination: 7

-- move 3 --
cups:  3  2 (5) 4  6  7  8  9  1 
pick up: 4, 6, 7
destination: 3

-- move 4 --
cups:  7  2  5 (8) 9  1  3  4  6 
pick up: 9, 1, 3
destination: 7

-- move 5 --
cups:  3  2  5  8 (4) 6  7  9  1 
pick up: 6, 7, 9
destination: 3

-- move 6 --
cups:  9  2  5  8  4 (1) 3  6  7 
pick up: 3, 6, 7
destination: 9

-- move 7 --
cups:  7  2  5  8  4  1 (9) 3  6 
pick up: 3, 6, 7
destination: 8

-- move 8 --
cups:  8  3  6  7  4  1  9 (2) 5 
pick up: 5, 8, 3
destination: 1

-- move 9 --
cups:  7  4  1  5  8  3  9  2 (6)
pick up: 7, 4, 1
destination: 5

-- move 10 --
cups: (5) 7  4  1  8  3  9  2  6 
pick up: 7, 4, 1
destination: 3

-- final --
cups:  5 (8) 3  7  4  1  9  2  6 
```

In the above example, the cups' values are the labels as they appear moving clockwise around the circle; the current cup is marked with ( ).

After the crab is done, what order will the cups be in? Starting after the cup labeled 1, collect the other cups' labels clockwise into a single string with no extra characters; each number except 1 should appear exactly once. In the above example, after 10 moves, the cups clockwise from 1 are labeled 9, 2, 6, 5, and so on, producing 92658374. If the crab were to complete all 100 moves, the order after cup 1 would be 67384529.

Using your labeling, simulate 100 moves. What are the labels on the cups after cup 1?

Your puzzle input is *364297581*.

In [1]:
#!fsharp
let testInput = "389125467"
let parse (input:string) =
    input.ToCharArray()
    |> Array.map (fun ch -> int ch - int '0')

parse "123450"

index,value
0,1
1,2
2,3
3,4
4,5
5,0


In [1]:
#!fsharp
let rec findDestination (buffer: int array) max cup =
    let index = 
        buffer 
        |> Array.tryFindIndex (fun f -> f = cup )
    match index with
    | Some index -> index
    | None -> findDestination buffer max (if cup=0 then max else cup-1)

findDestination [|3; 5; 4; 6; 7|] 9 1

4

In [1]:
#!fsharp
let step (buffer: int array) =
    let max = Array.max buffer
    let cup = buffer.[0]
    let rest = buffer.[4..]
    let destination = findDestination rest max cup

    Array.concat
        (seq {
            rest.[..destination]
            buffer.[1..3]
            rest.[(destination+1)..]
            buffer.[0..0]
        })

testInput |> parse |> step

index,value
0,2
1,8
2,9
3,1
4,5
5,4
6,6
7,7
8,3


In [1]:
#!fsharp
let toOutput (skipOne:bool) (input: int array) =
    let index = 
        input
        |> Array.findIndex (fun f -> f = 1)

    let start = if skipOne then index+1 else index
    Array.concat (seq {
        input.[start..]
        input.[..(index-1)]
    })
    |> Seq.map string
    |> String.concat ""

In [1]:
#!fsharp
let playGame rounds (input: int array) =
    let mutable buffer = input
    for i in 1..rounds do
        buffer <- step buffer
    buffer

testInput |> parse |> playGame 100


index,value
0,1
1,6
2,7
3,3
4,8
5,4
6,5
7,2
8,9


In [1]:
#!fsharp
testInput |> parse |> playGame 10 |> toOutput true

92658374

In [1]:
#!fsharp
testInput |> parse |> playGame 100 |> toOutput true

67384529

In [1]:
#!fsharp
let taskInput = "364297581"
taskInput |> parse |> playGame 100 |> toOutput true

47382659

--- Part Two ---

Due to what you can only assume is a mistranslation (you're not exactly fluent in Crab), you are quite surprised when the crab starts arranging many cups in a circle on your raft - one million (1000000) in total.

Your labeling is still correct for the first few cups; after that, the remaining cups are just numbered in an increasing fashion starting from the number after the highest number in your list and proceeding one by one until one million is reached. (For example, if your labeling were 54321, the cups would be numbered 5, 4, 3, 2, 1, and then start counting up from 6 until one million is reached.) In this way, every number from one through one million is used exactly once.

After discovering where you made the mistake in translating Crab Numbers, you realize the small crab isn't going to do merely 100 moves; the crab is going to do ten million (10000000) moves!

The crab is going to hide your stars - one each - under the two cups that will end up immediately clockwise of cup 1. You can have them if you predict what the labels on those cups will be when the crab is finished.

In the above example (389125467), this would be 934001 and then 159792; multiplying these together produces 149245887792.

Determine which two cups will end up immediately clockwise of cup 1. What do you get if you multiply their labels together?


In [1]:
#!fsharp
let fastPlayGame cups rounds (input:int array) =
    // next pointers in circular linked list. Note that we'll never use index 0
    let nextNodes = Array.init (cups+1) (fun index -> index+1 )
    let mutable currentCup = if input.Length>0 then input.[0] else 1

    if input.Length>0 then
        nextNodes.[cups] <- currentCup
    else
        nextNodes.[cups] <- 1

    for i in 0..(input.Length-1) do
        if i = input.Length - 1 then
            nextNodes.[input.[i]] <- if cups = input.Length then currentCup else input.Length+1
        else
            nextNodes.[input.[i]] <- input.[i+1]
    
    for i in 1..rounds do
        let mutable destination = currentCup-1
        if destination = 0 then destination <- cups
        let next1 = nextNodes.[currentCup]
        let next2 = nextNodes.[next1]
        let next3 = nextNodes.[next2]
        // no point optimizing further, it will loop at most 3 times
        while destination = next1 || destination = next2 || destination = next3 do
            destination <- destination - 1
            if destination = 0 then destination <- cups
        // first remove the 3 nodes
        nextNodes.[currentCup] <- nextNodes.[next3]
        // now insert the 3 nodes after destination
        nextNodes.[next3] <- nextNodes.[destination]
        nextNodes.[destination] <- next1
        currentCup <- nextNodes.[currentCup]
    
    currentCup <- nextNodes.[1]
    let result = Array.zeroCreate (cups-1)
    for i in 2..cups do
        result.[i-2] <- currentCup
        currentCup <- nextNodes.[currentCup]
    result


testInput |> parse |> fastPlayGame (testInput.Length+0) 100

index,value
0,6
1,7
2,3
3,8
4,4
5,5
6,2
7,9


In [1]:
#!fsharp
let getFinalResult (cups: int array) =
    printf "%i * %i" cups.[0] cups.[1]
    int64(cups.[0]) * int64(cups.[1])

testInput |> parse |> fastPlayGame 1000000 10000000 |> getFinalResult

934001

 * 

159792

149245887792

In [1]:
#!fsharp
taskInput |> parse |> fastPlayGame 1000000 10000000 |> getFinalResult

257760

 * 

163997

42271866720